## Deep Learning Course (980)
## Assignment Three 

__Assignment Goals:__

- Implementing RNN based language models.
- Implementing and applying a Recurrent Neural Network on text classification problem using TensorFlow.
- Implementing __many to one__ and __many to many__ RNN sequence processing.

In this assignment, you will implement RNN-based language models and compare extracted word representation from different models. You will also compare two different training methods for sequential data: Truncated Backpropagation Through Time __(TBTT)__ and Backpropagation Through Time __(BTT)__. 
Also, you will be asked to apply Vanilla RNN to capture word representations and solve a text classification problem. 


__DataSets__: You will use two datasets, an English Literature for language model task (part 1 to 4) and 20Newsgroups for text classification (part 5). 


1. (30 points) Implement the RNN based language model described by Mikolov et al.[1], also called __Elman network__ and train a language model on the English Literature dataset. This network contains input, hidden and output layer and is trained by standard backpropagation (TBTT with τ = 1) using the cross-entropy loss. 
   - The input represents the current word while using 1-of-N coding (thus its size is equal to the size of the vocabulary) and vector s(t − 1) that represents output values in the hidden layer from the previous time step. 
   - The hidden layer is a fully connected sigmoid layer with size 500. 
   - Softmax Output Layer to capture a valid probability distribution.
   - The model is trained with truncated backpropagation through time (TBTT) with τ = 1: the weights of the network are updated based on the error vector computed only for the current time step.
   
   Download the English Literature dataset and train the language model as described, report the model cross-entropy loss on the train set. Use nltk.word_tokenize to tokenize the documents. 
For initialization, s(0) can be set to a vector of small values. Note that we are not interested in the *dynamic model* mentioned in the original paper. 
To make the implementation simpler you can use Keras to define neural net layers, including Keras.Embedding. (Keras.Embedding will create an additional mapping layer compared to the Elman architecture.) 

2. (20 points) TBTT has less computational cost and memory needs in comparison with *backpropagation through time algorithm (BTT)*. These benefits come at the cost of losing long term dependencies [2]. Now let's try to investigate computational costs and performance of learning our language model with BTT. For training the Elman-type RNN with BTT, one option is to perform mini-batch gradient descent with exactly one sentence per mini-batch. (The input  size will be [1, Sentence Length]). 

    1. Split the document into sentences (you can use nltk.tokenize.sent_tokenize).
    2. For each sentence, perform one pass that computes the mean/sum loss for this sentence; then perform a gradient update for the whole sentence. (So the mini-batch size varies for the sentences with different lengths). You can truncate long sentences to fit the data in memory. 
    3. Report the model cross-entropy loss.

3. (15 points) It does not seem that simple recurrent neural networks can capture truly exploit context information with long dependencies, because of the problem that gradients vanish and exploding. To solve this problem, gating mechanisms for recurrent neural networks were introduced. Try to learn your last model (Elman + BTT) with the SimpleRnn unit replaced with a Gated Recurrent Unit (GRU). Report the model cross-entropy loss. Compare your results in terms of cross-entropy loss with two other approach(part 1 and 2). Use each model to generate 10 synthetic sentences of 15 words each. Discuss the quality of the sentences generated - do they look like proper English? Do they match the training set?
    Text generation from a given language model can be done using the following iterative process:
   1. Set sequence = \[first_word\], chosen randomly.
   2. Select a new word based on the sequence so far, add this word to the sequence, and repeat. At each iteration, select the word with maximum probability given the sequence so far. The trained language model outputs this probability. 

4. (15 points) The text describes how to extract a word representation from a trained RNN (Chapter 4). How we can evaluate the extracted word representation for your trained RNN? Compare the words representation extracted from each of the approaches using one of the existing methods.

5. (20 points) We are aiming to learn an RNN model that predicts document categories given its content (text classification). For this task, we will use the 20Newsgroupst dataset. The 20Newsgroupst contains messages from twenty newsgroups.  We selected four major categories (comp, politics, rec, and religion) comprising around 13k documents altogether. Your model should learn word representations to support the classification task. For solving this problem modify the __Elman network__ architecture such that the last layer is a softmax layer with just 4 output neurons (one for each category). 

    1. Download the 20Newsgroups dataset, and use the implemented code from the notebook to read in the dataset.
    2. Split the data into a training set (90 percent) and validation set (10 percent). Train the model on  20Newsgroups.
    3. Report your accuracy results on the validation set.

__NOTE__: Please use Jupyter Notebook. The notebook should include the final code, results and your answers. You should submit your Notebook in (.pdf or .html) and .ipynb format. (penalty 10 points) 

To reduce the parameters, you can merge all words that occur less often than a threshold into a special rare token (\__unk__).

__Instructions__:

The university policy on academic dishonesty and plagiarism (cheating) will be taken very seriously in this course. Everything submitted should be your own writing or coding. You must not let other students copy your work. Spelling and grammar count.

Your assignments will be marked based on correctness, originality (the implementations and ideas are from yourself), clarification and test performance.


[1] Tom´ as Mikolov, Martin Kara ˇ fiat, Luk´ ´ as Burget, Jan ˇ Cernock´ ˇ y,Sanjeev Khudanpur: Recurrent neural network based language model, In: Proc. INTERSPEECH 2010

[2] Tallec, Corentin, and Yann Ollivier. "Unbiasing truncated backpropagation through time." arXiv preprint arXiv:1705.08209 (2017).


In [1]:

"""This code is used to read all news and their labels"""
import os
import glob

def to_categories(name, cat=["politics","rec","comp","religion"]):
    for i in range(len(cat)):
        if str.find(name,cat[i])>-1:
            return(i)
    print("Unexpected folder: " + name) # print the folder name which does not include expected categories
    return("wth")

def data_loader(images_dir):
    categories = os.listdir(data_path)
    news = [] # news content
    groups = [] # category which it belong to
    
    for cat in categories:
        print("Category:"+cat)
        for the_new_path in glob.glob(data_path + '/' + cat + '/*'):
            news.append(open(the_new_path,encoding = "ISO-8859-1", mode ='r').read())
            groups.append(cat)

    return news, list(map(to_categories, groups))



data_path = "datasets/20news_subsampled"
news, groups = data_loader(data_path)

Category:talk.politics.mideast
Category:.DS_Store
Category:rec.autos
Category:comp.sys.mac.hardware
Category:rec.sport.baseball
Category:comp.os.ms-windows.misc
Category:rec.sport.hockey
Category:talk.politics.misc
Category:rec.motorcycles
Category:comp.windows.x
Category:comp.graphics
Category:comp.sys.ibm.pc.hardware
Category:talk.politics.guns
Category:soc.religion.christian
Category:talk.religion.misc


In [1]:
'''Implementing RNN based language models.
'''
from nltk import word_tokenize, download

# load the English Literature dataset
english_literature_path = './datasets/English Literature.txt'
with open(english_literature_path) as f:
    english_literature_text = f.read()
print(len(english_literature_text))

# tokenize the English Literature dataset
download('punkt')
english_literature_tokens = word_tokenize(english_literature_text)
print(len(english_literature_tokens))

1115394
[nltk_data] Downloading package punkt to /Users/songyih/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
254533


In [3]:
# build vocabulary
from collections import Counter

word2index = {}
index2word = []
english_literature_counter = Counter(english_literature_tokens)
for word, count in english_literature_counter.items():
    index2word.append(word)
    word2index[word] = len(word2index)

vocabulary_size = len(word2index)
print(vocabulary_size)

14309


In [20]:
# preprocess the dataset to get training data
from tensorflow.keras.utils import to_categorical
import numpy as np

max_input_len = 1
step = 1
x = []
y = []
for i in range(0, len(english_literature_tokens) - max_input_len, step):
    if i % 100 == 0:
        print("Progress: {0}%".format(round(i / len(english_literature_tokens) * 100, 2)), end="\r")
    curr_words = english_literature_tokens[i:i + max_input_len]
    x.append([word2index.get(curr_word, 0) for curr_word in curr_words])
    next_word = english_literature_tokens[i + max_input_len]
    y.append(word2index.get(next_word, 0))
X = np.array(x)
Y = to_categorical(y, vocabulary_size)
print("")
print(X.shape, Y.shape)

Progress: 99.99%
(254532, 1) (254532, 14309)


In [21]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

model_elman = Sequential()
model_elman.add(Embedding(vocabulary_size, 500, input_length=max_input_len))
model_elman.add(SimpleRNN(units=500))
model_elman.add(Dense(vocabulary_size, activation='softmax'))

model_elman.summary()
model_elman.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 1, 500)            7154500   
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 500)               500500    
_________________________________________________________________
dense_3 (Dense)              (None, 14309)             7168809   
Total params: 14,823,809
Trainable params: 14,823,809
Non-trainable params: 0
_________________________________________________________________


In [22]:
from tensorflow.python.client import device_lib
from tensorflow.keras.callbacks import ModelCheckpoint

print(device_lib.list_local_devices())
best_model_checkpoint = ModelCheckpoint('./best_model_elman.pth', monitor="val_acc", save_best_only=True, save_weights_only=False)
elman_training_history = model_elman.fit(
    X,
    Y,
    batch_size=1024,
    epochs=10,
    verbose=1,
    shuffle=False,
    callbacks=[best_model_checkpoint]
)


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17995072832655268474
]
Epoch 1/10
254532/254532 [==============================] - 180s 707us/sample - loss: 6.4290 - acc: 0.1062
Epoch 2/10
254532/254532 [==============================] - 178s 701us/sample - loss: 5.6275 - acc: 0.1432
Epoch 3/10
254532/254532 [==============================] - 178s 699us/sample - loss: 5.4408 - acc: 0.1543
Epoch 4/10
254532/254532 [==============================] - 174s 686us/sample - loss: 5.3278 - acc: 0.1601
Epoch 5/10
254532/254532 [==============================] - 178s 701us/sample - loss: 5.2453 - acc: 0.1659
Epoch 6/10
254532/254532 [==============================] - 176s 690us/sample - loss: 5.1788 - acc: 0.1705
Epoch 7/10
254532/254532 [==============================] - 178s 699us/sample - loss: 5.1227 - acc: 0.1747
Epoch 8/10
254532/254532 [==============================] - 180s 706us/sample - loss: 5.0745 - acc: 0.1789
Epoch 9/10
254532/254532 [===

In [ ]:
from nltk.tokenize import sent_tokenize
